In [1]:
from typing import Tuple, List, Dict, Any, Sequence
import polars as pl
import polars.selectors as cs
import pandas as pd
from pprint import pprint, pformat
import sys
from pathlib import Path
from tqdm import tqdm, trange
import psutil
import re


# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.utils import process_lazyframe_in_chunks
from code.loading import DataLoader
from code.preprocess import TextPreprocessor, create_udi_preprocessor, create_company_preprocessor, create_generic_preprocessor
from code.preprocess.preprocess import get_pattern_cols, \
    get_unique_by_cols_safe, get_unique, \
    analyze_null_values, replace_pattern_with_null, overview_col

## 원본 데이터 불러오기

In [2]:
# maude 데이터 불러오기
loader1 = DataLoader(
    start=2020,
    end=2025,
    output_file = DATA_DIR / 'maude_sample.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
maude_lf = loader1.load(adapter=adapter, **polars_kwargs)
maude_lf


📖 /Users/eric/yeeun/Project4/data/maude_sample.parquet 로딩 중... (adapter=polars)


In [3]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=DATA_DIR/'udi.parquet',
)

udi_lf = udi_loader.load(adapter, **polars_kwargs)
udi_lf


📖 /Users/eric/yeeun/Project4/data/udi.parquet 로딩 중... (adapter=polars)


## UDI Dataset 전처리

In [4]:
IDENTIFIER_PATTERNS = [
    r"^device_\d+_brand_name$",
    r"identifiers_\d+_id", 
    r"identifiers_\d+_issuing_agency", 
    r"identifiers_\d+_package_discontinue_date", 
    r"identifiers_\d+_package_status", 
    r"identifiers_\d+_package_type", 
    r"identifiers_\d+_quantity_per_package", 
    r"identifiers_\d+_type", 
    r"identifiers_\d+_unit_of_use_id"
]
UDI_DI_PATTERNS = [r'^identifiers_\d+_id$']
TYPE_PATTERNS = [
    r'identifiers_\d+_type'
]

CUSTOMER_PATTERNS = [r'^customer']
DEVICE_SIZE_PATTERNS = [r'^device_sizes']
STORAGE_PATTERNS = [r'^storage']

### Drop 필요없는 열

In [5]:
drop_patterns = CUSTOMER_PATTERNS + DEVICE_SIZE_PATTERNS + STORAGE_PATTERNS

regex = "|".join(drop_patterns)

udi_lf = udi_lf.select(
    ~cs.matches(regex)
)

# udi_lf.collect_schema().names()

### Primary 추출

In [6]:
udi_di_cols = get_pattern_cols(udi_lf, UDI_DI_PATTERNS)
identifiers_cols = get_pattern_cols(udi_lf, IDENTIFIER_PATTERNS)
type_cols = get_pattern_cols(udi_lf, TYPE_PATTERNS)

In [7]:
# type-udi_di 쌍 만들기 (인덱스로 매칭)
def extract_index(col_name):
    match = re.search(r'identifiers_(\d+)_', col_name)
    return int(match.group(1)) if match else None

type_id_pairs = []
for type_col in type_cols:
    idx = extract_index(type_col)
    udi_di_col = f'identifiers_{idx}_id'
    if udi_di_col in udi_di_cols:
        type_id_pairs.append((type_col, udi_di_col))

len(type_id_pairs)

132

In [8]:
primary_udi_unique = set()

for type_col, id_col in tqdm(type_id_pairs, desc="Processing columns", unit="col"):
    try:
        count = (
            udi_lf
            .filter(pl.col(type_col).eq("Primary"))
            .select(pl.len())
            .collect()
            .item()
        )
        
        if count > 0:
            ids = (
                udi_lf
                .filter(pl.col(type_col).eq("Primary"))
                .select(pl.col(id_col))
                .unique()
                .collect()
                .to_series()
                .drop_nulls()
                .to_list()
            )
            primary_udi_unique.update(ids)
            
    except Exception as e:
        tqdm.write(f"Error processing {type_col}: {e}")
        continue

print(f"\n{'='*50}")
print(f"UDI 데이터의 고유 Primary udi 개수: {len(primary_udi_unique):,}")
print(f"{'='*50}")

Processing columns: 100%|██████████| 132/132 [00:00<00:00, 151.17col/s]


UDI 데이터의 고유 Primary udi 개수: 4,903,764


In [9]:
udi_step1_path = DATA_DIR / 'silver' / 'udi_primary.parquet'
udi_step2_path = DATA_DIR / 'silver' / 'udi_clean.parquet'

In [10]:
# # Primary인 id를 추출
# def primary_transform(lf: pl.LazyFrame):
#     return lf.with_columns(
#         pl.coalesce([
#             pl.when(pl.col(type_col).eq("Primary"))
#             .then(pl.col(id_col))
#             for type_col, id_col in type_id_pairs
#         ]).alias('primary_udi_di')
#     )

# process_lazyframe_in_chunks(
#     udi_lf, 
#     primary_transform, 
#     udi_step1_path, 
#     10_000,
#     desc="Primary extraction"
# )

In [11]:
# # udi 데이터 불러오기
# udi_loader = DataLoader(
#     name='udi',
#     output_file=udi_step1_path,
# )

# primary_udi_lf = udi_loader.load(adapter, **polars_kwargs)

In [12]:
# # 회사 이름 정제
# preprocessor1 = create_company_preprocessor()

# preprocessor1.apply_to_lazyframe(
#     primary_udi_lf,
#     ['company_name', 'brand_name'],
#     udi_step2_path,
#     10_000,
# )

# del preprocessor1, primary_udi_lf
# udi_step1_path.unlink(missing_ok=True)

In [13]:
# udi 데이터 불러오기
udi_loader = DataLoader(
    name='udi',
    output_file=udi_step2_path,
)

cleaned_udi_lf = udi_loader.load(adapter, **polars_kwargs)


📖 /Users/eric/yeeun/Project4/data/silver/udi_clean.parquet 로딩 중... (adapter=polars)


In [14]:
# maude_lf.filter(
#     pl.col('device_0_udi_di').is_not_null()
# ).group_by(
#     ['device_0_manufacturer_d_name', 'device_0_brand_name', 'device_0_model_number', 'device_0_catalog_number']
# ).agg(
#     pl.col('device_0_udi_di').n_unique().alias('udi_nunique'),
#     pl.col('device_0_udi_di').unique().alias('udi_unique'),
#     pl.col('device_0_udi_di').mode().alias('udi_mode')
# ).sort('udi_nunique', descending=True).head(10).collect().to_pandas()

## MAUDE 1차 전처리

In [15]:
# 기본 변수
BASE_COLS = [
    'mdr_report_key', 'report_number', 'adverse_event_flag', 'product_problem_flag', 
    'date_of_event', 'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'report_to_fda', 'event_location', 
    'manufacturer_link_flag', 'manufacturer_g1_name', 'manufacturer_g1_postal_code',
    'pma_pmn_number'
]

DEVICE_COLS = [
    "device_0_manufacturer_d_name",
    "device_0_manufacturer_d_postal_code",
    "device_0_brand_name",
    "device_0_catalog_number",
    "device_0_model_number",
    "device_0_udi_di",
    "device_0_lot_number",
    "device_0_udi_public",
    "device_0_device_report_product_code",
    "device_0_device_age_text",
    "device_0_device_operator",
    "device_0_openfda_device_class",
    "device_0_openfda_device_name",
]



PATIENT_COLS = [
    "patient_0_patient_sequence_number",
    "patient_0_patient_age",
    "patient_0_patient_sex",
    "patient_0_patient_weight",
    "patient_0_patient_race",
    "patient_0_patient_problems",
    "patient_0_sequence_number_outcome",
    "patient_0_sequence_number_treatment",
]



MDR_TEXT_PATTERNS = [
    r"^mdr_text_.*_text$",
    r"^mdr_text_.*_text_type_code$",
]

MDR_COLS = get_pattern_cols(maude_lf, MDR_TEXT_PATTERNS)
TOTAL_COLS = BASE_COLS + DEVICE_COLS + PATIENT_COLS + MDR_COLS

### Drop 필요 없는 열

In [16]:
maude_lf = maude_lf.select(TOTAL_COLS)

# maude_lf.collect_schema().names()

In [17]:
maude_step1_path = DATA_DIR / 'silver' / 'clean_step1.parquet'
maude_step2_path = DATA_DIR / 'silver' / 'clean_step2.parquet'
maude_step3_path = DATA_DIR / 'silver' / 'clean_step3.parquet'

In [18]:
# # 1단계
# preprocessor1 = create_udi_preprocessor()
# preprocessor1.apply_to_lazyframe(
#     maude_lf, 'device_0_udi_di', maude_step1_path, chunk_size=10_000
# )
# del preprocessor1  # 명시적 삭제

In [19]:
# # 2단계
# maude_lf2 = pl.scan_parquet(maude_step1_path)
# preprocessor2 = create_company_preprocessor()
# preprocessor2.apply_to_lazyframe(
#     maude_lf2, ['device_0_manufacturer_d_name', 'manufacturer_g1_name', 'device_0_brand_name'], 
#     maude_step2_path, chunk_size=10_000
# )
# del maude_lf2, preprocessor2

In [20]:
# # 중간 파일 정리
# maude_step1_path.unlink(missing_ok=True)

# # 3단계
# maude_lf3 = pl.scan_parquet(maude_step2_path)
# preprocessor3 = create_generic_preprocessor()
# preprocessor3.apply_to_lazyframe(
#     maude_lf3, ['device_0_model_number', 'device_0_catalog_number', 'device_0_lot_number'], 
#     maude_step3_path, chunk_size=10_000
# )
# del maude_lf3, preprocessor3

In [21]:
maude_step2_path.unlink(missing_ok=True)

# maude 데이터 불러오기
loader3 = DataLoader(
    start=2020,
    end=2025,
    output_file = maude_step3_path,
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
    # 'n_rows': 1000
}
cleaned_maude_lf = loader3.load(adapter=adapter, **polars_kwargs)
cleaned_maude_lf


📖 /Users/eric/yeeun/Project4/data/silver/clean_step3.parquet 로딩 중... (adapter=polars)


## 작업에 필요한 컬럼

In [22]:
rename_udi_lf = cleaned_udi_lf.rename({
    'company_name': 'manufacturer',
    'brand_name': 'brand',
    'version_or_model_number': 'model_number',
    'primary_udi_di': 'udi_di',
})

rename_maude_lf = cleaned_maude_lf.rename({
    'device_0_manufacturer_d_name': 'manufacturer',
    'device_0_brand_name': 'brand',
    'device_0_model_number': 'model_number',
    'device_0_catalog_number': 'catalog_number',
    'device_0_lot_number': 'lot_number',
    'device_0_udi_di': 'udi_di',
    'device_0_udi_public': 'udi_public'
})

In [23]:
target_cols = [
    'manufacturer',
    'brand',
    'model_number',
    'catalog_number'
]

join_col = 'udi_di'

common_cols = target_cols + [join_col]

maude_cols = common_cols + [
    'mdr_report_key',
]

udi_cols = common_cols + udi_di_cols

In [24]:
udi_necessary_lf = rename_udi_lf.select(pl.col(udi_cols))
maude_necessary_lf = rename_maude_lf.select(pl.col(maude_cols))

## 고유값 추출

In [25]:
cols_group = {
    'udi': udi_di_cols,
}

# udi 데이터셋의 udi_di 고유값
udi_udi_unique = get_unique_by_cols_safe(
    udi_necessary_lf, 
    cols_group,
    memory_safety_ratio=0.3,
    calibration_factor = 1
)['udi']

문자열 길이 통계를 실제 데이터로부터 추정 중...
  문자열 길이 통계:
    - 평균(mean): 13.9자
    - 중앙값(median): 14.0자
    - 75%ile: 14.0자
    - 90%ile: 14.0자
    - 최댓값: 23자
  → 사용할 크기(p75): 14.0자

=== 메모리 기반 자동 임계값 계산 ===
사용 가능한 메모리: 24.62 GB
안전 사용 메모리 (30%): 7.39 GB
예상 바이트/고유값: 106 bytes
  - 문자열 데이터: 28 bytes
  - str 오버헤드: 50 bytes
  - set 오버헤드: 28 bytes
계산된 최대 고유값: 74,809,112개



Extracting unique values:   0%|          | 0/1 [00:00<?, ?it/s]

udi: 6,336,862개의 고유값 (예상 메모리: 640.6 MB)


Extracting unique values: 100%|██████████| 1/1 [00:03<00:00,  3.69s/it]

  ✓ udi 추출 완료 (실제 메모리: 636.11 MB)
    → 예상치 정확도: 100.7% (예상/실제 비율)


=== 추출 요약 ===
성공: 1/1
실패/스킵: 0/1


In [26]:
maude_udi_unique = get_unique(maude_necessary_lf, ['udi_di'])
angry_udi_unique = maude_udi_unique - udi_udi_unique
survive_udi_unique = maude_udi_unique & udi_udi_unique

print(f'UDI 데이터의 고유 udi 개수: {len(udi_udi_unique)}개')
print(f'MAUDE 데이터의 고유 udi 개수: {len(maude_udi_unique)}개')
print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(angry_udi_unique)}개')
print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(survive_udi_unique)}개')

UDI 데이터의 고유 udi 개수: 6336861개
MAUDE 데이터의 고유 udi 개수: 59072개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 10593개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 48479개


In [27]:
maude_primary_udi_unique = survive_udi_unique & primary_udi_unique
maude_secondary_udi_unique = survive_udi_unique - primary_udi_unique

print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(maude_primary_udi_unique)}개')
print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(maude_secondary_udi_unique)}개')

Primary UDI인 MAUDE 데이터의 고유 udi 개수: 46114개
Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: 2365개


In [28]:
class UniqueUDIDI:
    def __init__(
        self,
        udi_udi_unique: set = None,
        maude_udi_unique: set = None,
        angry_udi_unique: set = None,
        survive_udi_unique: set = None,
        primary_udi_unique: set = None,
        maude_primary_udi_unique: set = None,
        maude_secondary_udi_unique: set = None
    ):
        self.udi = udi_udi_unique
        self.maude = maude_udi_unique
        self.angry = angry_udi_unique
        self.survive = survive_udi_unique
        self.primary = primary_udi_unique
        self.maude_primary = maude_primary_udi_unique
        self.maude_secondary = maude_secondary_udi_unique

    def print_stats(self):
        print(f'UDI 데이터의 고유 udi 개수: {len(self.udi)}개')
        print(f'UDI 데이터의 고유 Primary udi 개수: {len(self.primary)}개')
        print(f'MAUDE 데이터의 고유 udi 개수: {len(self.maude)}개')
        print(f'UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: {len(self.survive)}개')
        print(f'Primary UDI인 MAUDE 데이터의 고유 udi 개수: {len(self.maude_primary)}개')
        print(f'Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: {len(self.maude_secondary)}개')
        print(f'UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: {len(self.angry)}개')

unique_udi_di = UniqueUDIDI(udi_udi_unique, maude_udi_unique, angry_udi_unique, survive_udi_unique, primary_udi_unique, maude_primary_udi_unique, maude_secondary_udi_unique)

unique_udi_di.print_stats()

UDI 데이터의 고유 udi 개수: 6336861개
UDI 데이터의 고유 Primary udi 개수: 4903764개
MAUDE 데이터의 고유 udi 개수: 59072개
UDI 데이터에 있는 MAUDE 데이터의 고유 udi 개수: 48479개
Primary UDI인 MAUDE 데이터의 고유 udi 개수: 46114개
Primary UDI가 아닌 MAUDE 데이터의 고유 udi 개수: 2365개
UDI 데이터에 없는 MAUDE 데이터의 고유 udi 개수: 10593개


## UDI 데이터셋에 Primary 컬럼 생성

In [29]:
# MAUDE UDI를 LazyFrame으로
maude_udi_lf = pl.LazyFrame({
    'udi_di': list(unique_udi_di.maude_secondary)
})

In [30]:
# unpivot 후 즉시 필터링 (메모리 증가 최소화)
udi_mapping_lf = (
    udi_necessary_lf
    .with_row_index('row_idx')
    .unpivot(
        index=['row_idx'] + common_cols,
        on=udi_di_cols,
        variable_name='matched_col',
        value_name='match_udi_di'
    )
    .filter(
        pl.col('match_udi_di').is_not_null() &  # null 제거
        pl.col('match_udi_di').is_in(unique_udi_di.maude_secondary)  # 매칭되는 것만
    )
    .unique(subset=['match_udi_di', 'row_idx'])
    .select(['match_udi_di', 'row_idx'] + common_cols)
)

In [31]:
# udi_mapping_df = udi_mapping_lf.collect().to_pandas()

In [32]:
# # 통계만 먼저 확인
# print(f"Total matches: {len(udi_mapping_df):,}")
# print(f"Unique UDIs: {udi_mapping_df['match_udi_di'].nunique():,}")

In [33]:
# udi_mapping_df[udi_mapping_df['match_udi_di'] == '00021292007706']

In [34]:
maude_necessary_lf.filter(
    pl.col('udi_di').eq('00021292007706')
).head(15).collect().to_pandas().transpose()

,0,1
manufacturer,TRIVIDIA HEALTH,TRIVIDIA HEALTH
brand,TRUE METRIX GO,TRUE METRIX GO
model_number,KIT GO TVH TMX METERMG/DL,KIT GO CVS TMX METERMG/DL
catalog_number,None,None
udi_di,00021292007706,00021292007706
mdr_report_key,20041268,18722016


# 전처리 함수 설계

In [35]:
# 일치 점수 매기는 함수
# 1. 매핑한 행만 불러오기
# 2. company_name, brand_name, version_or_mode_number, catalog_number에 대하여 
# 2-1. 전부 대문자로
# 2-2. 일치 점수 매기기
# 2-3. 가장 점수가 높은 것의 primary_udi_di를 할당
# 2-4. 필요한 정보 가져오기

# udi를 깔끔하게 만듦
# primary: 그대로 primary_lf와 매칭해서 필요 info 가져옴
# secondary: primary_lf 중 udi 매칭한 lf에서 점수 높은 info 가져옴
# null: primary_lf에서 점수 높은 info 가져옴

### primary와 매칭해서 가져오는 함수

In [36]:
def extract_from_match(
    src_lf: pl.LazyFrame, desc_lf: pl.LazyFrame, 
    on: str | Sequence[str], 
    target_cols: str | Sequence[str]
):
    if isinstance(on, str):
        on = [on]
    
    if isinstance(target_cols, str):
        target_cols = [target_cols]
    
    udi_subset = src_lf.select(pl.col(on + target_cols))
    
    result = desc_lf.join(
        udi_subset,
        on=on,
        how='left'
    )
    
    for col in target_cols:
        result = result.with_columns(
            pl.coalesce([f'{col}_right', col]).alias(col)
        ).drop(f'{col}_right')
    
    return result

maude_match_lf = extract_from_match(udi_necessary_lf, rename_maude_lf, join_col, target_cols)

# cleaned_maude_lf.head(10).collect().to_pandas()
maude_match_lf.head(10).collect().to_pandas()

,mdr_report_key,report_number,adverse_event_flag,product_problem_flag,date_of_event,date_received,device_date_of_manufacturer,event_type,previous_use_code,single_use_flag,...,mdr_text_5_text,mdr_text_5_text_type_code,mdr_text_6_text,mdr_text_6_text_type_code,mdr_text_7_text,mdr_text_7_text_type_code,mdr_text_8_text,mdr_text_8_text_type_code,mdr_text_9_text,mdr_text_9_text_type_code
0,19448579,3013756811-2024-97562,N,Y,20240510,20240603,20210201,Malfunction,R,N,...,None,None,None,None,None,None,None,None,None,None
1,19448674,8030965-2024-07013,N,Y,20240101,20240603,None,Malfunction,U,N,...,None,None,None,None,None,None,None,None,None,None
2,19448702,3004753838-2024-130855,N,Y,20240504,20240603,20240101,Malfunction,I,Y,...,None,None,None,None,None,None,None,None,None,None
3,19448836,3013756811-2024-88985,N,Y,20240510,20240603,20221201,Malfunction,R,N,...,None,None,None,None,None,None,None,None,None,None
4,19448850,9612169-2024-00511,N,Y,None,20240603,20231117,Malfunction,I,None,...,None,None,None,None,None,None,None,None,None,None
5,19448931,3004753838-2024-130890,N,Y,20240506,20240603,20230524,Malfunction,I,N,...,None,None,None,None,None,None,None,None,None,None
6,19449093,3013756811-2024-93158,N,Y,20240509,20240603,20221101,Malfunction,R,N,...,None,None,None,None,None,None,None,None,None,None
7,19449660,2032227-2024-181093,N,Y,20240513,20240603,20230620,Malfunction,A,N,...,None,None,None,None,None,None,None,None,None,None
8,19449741,2518422-2024-16641,Y,Y,20231012,20240603,20160309,Injury,None,N,...,None,None,None,None,None,None,None,None,None,None
9,19613702,3004753838-2024-151007,N,Y,20240508,20240626,20220425,Malfunction,I,Y,...,None,None,None,None,None,None,None,None,None,None


In [37]:
analyze_null_values(maude_match_lf, common_cols)
analyze_null_values(rename_maude_lf, common_cols)
overview_col(maude_match_lf, 'brand', n_rows=100)
overview_col(rename_maude_lf, 'brand', n_rows=100)
overview_col(maude_lf, 'device_0_brand_name', n_rows=1000)


=== 결측치 분석 ===
전체 행 수: 2,627,150

udi_di                                       :    975,101개 ( 37.12%)
model_number                                 :    476,339개 ( 18.13%)
catalog_number                               :    315,278개 ( 12.00%)
brand                                        :     47,292개 (  1.80%)
manufacturer                                 :      3,065개 (  0.12%)

=== 결측치 분석 ===
전체 행 수: 2,627,150

model_number                                 :  1,157,071개 ( 44.04%)
udi_di                                       :    975,101개 ( 37.12%)
catalog_number                               :    382,838개 ( 14.57%)
brand                                        :     48,248개 (  1.84%)
manufacturer                                 :      5,591개 (  0.21%)
brand의 고유 개수: 47189


,head_brand,tail_brand
0,None,ZMAX IMPLANT SYSTEM
1,0 KEEL LEFT SIZE D CEMENTED TIBIA,ZMED II
2,0 KEEL TIBIAL PEG DRILL,ZMED II CATHETER
3,0 MM ML TAPER RASP SZ 4,ZMR
4,0 MM RASP LONG POST SIZE 4,ZNN CMN LAG SCREW REAMER SHORT
...,...,...
95,10MM TI CANN FRN GT 340MM RIGHT STERILE,ZYNO PUMPS
96,10MM TI CANN FRN GT 360MM LEFT STERILE,ZYPHR
97,10MM 3FLUTED ACORN REAMER,ZYPHR DISP CRAN PERF LRG 1411
98,10MM BLUNT BLADE AND TUBE SET,ZYSTON CURVE SPACER SYS


brand의 고유 개수: 61602


,head_brand,tail_brand
0,None,ZOLL CARDIAC MONITOR
1,0 ENDOLOOP LIG WPDS II,ZOLL COOLGARD 3000 IVTM SYSTEM
2,0 KEEL LEFT SIZE D CEMENTED TIBIA,ZOLL DEFIBRILLATOR
3,0 KEEL LEFT SIZE G CEMENTED TIBIA,ZOLL DEFIBRILLATOR PADS
4,0 KEEL RIGHT SIZE C CEMENTED TIBIA,ZOLL DEFIBRILLATOR PATCHES
...,...,...
95,100 THIN SWIVEL DIRECT FLOW 30K,ZYNO MEDICAL Z800WF INFUSION PUMP SYSTEM
96,1000 TRIPLE BEND SWIVEL DIRECT FLOW 25K,ZYNO PUMP
97,1000 TRIPLE BEND SWIVEL DIRECT FLOW ULTRASONIC...,ZYNO PUMPS
98,1000ML PLEURX SPARE BOTTLE LATAM,ZYPHR DISP CRAN PERF LRG 1411


device_0_brand_name의 고유 개수: 66169


,head_device_0_brand_name,tail_device_0_brand_name
0,None,WHITE JET ROUTINE I TAPED CASSETTE NO/LID
1,"""1.5MM"" SYSTEM TW DRILL 1.1X50MM 15MMSTP W/NT",WHITE JET ROUTINE III TAPED CASSETTE W/LID
2,"""COMFORT, COMFORT SHORT, SILHOUETTE, NERIA SOF...",WHITE PEARL ANTERIOR CERVICAL PLATE
3,"""GORE- VIATORRI-TIPS ENDOPROSTHESIS WITH CONTR...",WHITE RELOAD FOR ECHELON 45
4,"""SILHOUETTE PARADIGM",WHITE RELOAD FOR ECHELON 60
...,...,...
995,2.4MM LCP(TM) T-PLATE 2 HOLES HEAD/7 HOLES SHAFT,"Ø8X 20MM BC IF SCRW, VENTED"
996,2.4MM LOCKING SCREW SELF-TAPPING-20MM,"Ø8X 30MM BC IF SCRW, VENTED"
997,2.4MM LOCKING SCREW SLF-TPNG WITH STARDRIVE RE...,"Ø9X 20MM BC IF SCRW, VENTED"
998,2.4MM LOCKING SCREW SLF-TPNG WITH STARDRIVE RE...,"Ø9X 30MM BC IF SCRW, VENTED"


In [38]:
import polars as pl
from pathlib import Path
from code.preprocess.preprocessor import UDIProcessor
from code.preprocess.config import Config


output_path=DATA_DIR / "maude_with_udi.parquet"

# 처리
processor = UDIProcessor(Config())
result_path = processor.process(
    maude_lf=rename_maude_lf,
    udi_lf=rename_udi_lf,
    output_path=Path(output_path),
    chunk_size=10_000
)


UDI 처리 파이프라인 시작 (Path 기반)
🔧 MAUDE 전처리...
   ✓ 전처리 완료
🔧 UDI DB 전처리...
🔧 제조사명 퍼지 매칭...
   매칭: 359/3910 건
🔧 Lookup 테이블 생성...
   Primary UDI Lookup: 4,903,764 건
   Secondary 컬럼: 264개
   Full UDI Lookup: LazyFrame
🔧 UDI 매핑 테이블 생성 (Score 기반)...
   Unique UDI: 59,114 건
   - Primary 직접 매칭: 59,114 건
   - Secondary 매칭 시도: 0 건
   - Secondary 매칭 성공: 0 건
   - No UDI 매칭 시도: 69,956 건
      No UDI 매칭 (Path 기반)...
Processing 4,903,764 rows in chunks of 10,000...


No-UDI 제조사 필터: 100%|██████████| 491/491 [00:01<00:00, 300.88it/s]


Merging chunks...
✓ Saved to /Users/eric/yeeun/Project4/data/temp/no_udi_lookup_3f3564aa4c6b4c699bf112c80b166469.parquet
Processing 69,956 rows in chunks of 10,000...


No-UDI score match:   0%|          | 0/7 [00:00<?, ?it/s]

Score >= 3 → 231 건 성공
Score >= 2 → 631 건 성공
Score >= 1 → 1,243 건 성공


...

Merging chunks...
✓ Saved to /Users/eric/yeeun/Project4/data/temp/no_udi_matched_0d5c867fbc224529a8a724fdda9f370b.parquet
   - No UDI 매칭 성공: 18,531 건
   ✅ 최종 UDI 매핑: 146,082 건
shape: (3, 2)
┌────────────────┬───────┐
│ udi_match_type ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ no_match       ┆ 68437 │
│ udi_direct     ┆ 59114 │
│ meta_match     ┆ 18531 │
└────────────────┴───────┘

🔧 매칭 적용 중...
Processing 2,627,150 rows in chunks of 10,000...


UDI 매핑 적용: 100%|██████████| 263/263 [00:17<00:00, 15.27it/s]


Merging chunks...
✓ Saved to /Users/eric/yeeun/Project4/data/temp/maude_matched.parquet

🔧 후처리 (Tier 3)...
Processing 2,741,510 rows in chunks of 10,000...


Tier 3 처리: 100%|██████████| 275/275 [00:10<00:00, 26.08it/s]


Merging chunks...
✓ Saved to /Users/eric/yeeun/Project4/data/temp/resolved_final.parquet
✅ 최종 결과: /Users/eric/yeeun/Project4/data/temp/resolved_final.parquet

📊 최종 결과

매칭 출처 분포:
shape: (4, 3)
┌────────────────┬─────────┬─────────┐
│ match_source   ┆ count   ┆ percent │
│ ---            ┆ ---     ┆ ---     │
│ str            ┆ u32     ┆ f64     │
╞════════════════╪═════════╪═════════╡
│ udi_direct     ┆ 1679222 ┆ 61.25   │
│ not_in_mapping ┆ 572206  ┆ 20.87   │
│ no_match       ┆ 395336  ┆ 14.42   │
│ meta_match     ┆ 94746   ┆ 3.46    │
└────────────────┴─────────┴─────────┘

신뢰도 분포:
shape: (3, 3)
┌────────────────┬─────────┬─────────┐
│ udi_confidence ┆ count   ┆ percent │
│ ---            ┆ ---     ┆ ---     │
│ str            ┆ u32     ┆ f64     │
╞════════════════╪═════════╪═════════╡
│ HIGH           ┆ 1679222 ┆ 61.25   │
│ VERY_LOW       ┆ 967542  ┆ 35.29   │
│ MEDIUM         ┆ 94746   ┆ 3.46    │
└────────────────┴─────────┴─────────┘

Score 분포:
shape: (5, 3)
┌─────────────┬────

In [ ]:
loader4 = DataLoader(
    name='event',
    output_file=output_path
)

matched_lf = loader4.load(adapter=adapter, **polars_kwargs)

## 컬럼 선택 및 행 Drop

In [ ]:
matched_lf.select('mdr_report_key')

In [ ]:
matched_lf.filter(
    pl.col('final_source').eq('not_in_mapping')
    # & pl.col('udi_matched').eq(False)
).head(10).collect().to_pandas().transpose()

In [ ]:
matched_lf.filter(
    pl.col('device_version_id').is_null()
    & pl.col('manufacturer_final').is_null()
).head(10).collect().to_pandas().transpose()

In [ ]:
overview_col(rename_maude_lf, 'brand')

In [ ]:
# overview_col(matched_lf, 'device_version_id')
overview_col(cleaned_maude_lf, 'date_of_event')

### 결측 패턴 분석

In [ ]:
# import polars as pl

# # 결측 패턴 분석 (Polars LazyFrame 버전)
# missing_pattern = {
#     'overall_rate': (
#         maude_match_lf
#         .select(pl.col('udi_di').is_null().mean())
#         .collect()
#         .item()
#     ),
    
#     'by_manufacturer': (
#         maude_match_lf
#         .group_by('manufacturer')
#         .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
#         .collect()
#     ),
    
#     'by_year': (
#         maude_match_lf
#         .with_columns(pl.col('date_received').str.to_datetime(format='%Y%m%d').dt.year().alias('year'))
#         .group_by('year')
#         .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
#         .collect()
#     ),
    
#     'by_product_code': (
#         maude_match_lf
#         .group_by('product_code')
#         .agg(pl.col('udi_di').is_null().mean().alias('missing_rate'))
#         .collect()
#     )
# }

In [ ]:
zero_brand = rename_maude_lf.filter(
    pl.col('brand').eq('00421871')
).head(10).collect()

zero_brand.to_pandas().transpose()

## MAUDE 데이터의 UDI를 깔끔하게

In [ ]:
group_cols = ['manufacturer', 'brand','model_number', 'lot_number']

In [ ]:
group_lf = matched_lf.select(['udi_di', *group_cols]).group_by(group_cols).agg(
    pl.col('udi_di').n_unique().alias('udi_di_unique'),
)

outlier = group_lf.filter(
    pl.col('udi_di_unique').gt(1)
).select(pl.len()).collect().item()

print(f'UDI-DI 이상치는 {outlier}개 있습니다.')
group_lf.collect().drop_nulls().sort('udi_di_unique', descending=True).head(100).to_pandas()

In [ ]:
rename_maude_lf.select(group_cols + ['udi_di']).filter(
    pl.col('manufacturer').eq('DEXCOM'),
    pl.col('model_number').eq('9500-161'),
    pl.col('brand').eq('DEXCOM G7 CONTINUOUS GLUCOSE MONITORING SYSTEM'),
).drop_nulls().head(1000).collect().to_pandas()